In [5]:
import numpy as np
import pandas as pd
import requests
import io
from scipy import optimize
import matplotlib.pyplot as plt
import requests, pandas as pd
from bs4 import BeautifulSoup
import urllib.request

In [2]:
url='https://en.wikipedia.org/wiki/List_of_urban_areas_in_the_United_Kingdom#cite_note-6'

In [3]:
tables=pd.read_html('https://en.wikipedia.org/wiki/List_of_urban_areas_in_the_United_Kingdom#cite_note-6')

In [10]:
page = urllib.request.urlopen(url)
soup = BeautifulSoup(page, "lxml")


In [30]:
all_tables=soup.find_all("table")

right_table=soup.find_all("table", class_="wikitable sortable")

TypeError: list indices must be integers or slices, not list

In [39]:
Number=[]
Area=[]
Population=[]
Area_size=[]
Density=[]
Primary_sub_division=[]
Notable_changes_from_2001_to_2011=[]

for row in right_table[1].findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==5:
        Area.append(cells[0].find(text=True))
        Alnk=cells[1].findAll('a')
        Population.append(cells[1].find(text=True))
        Area_size.append(cells[2].find(text=True))
        Density.append(cells[3].find(text=True))
        Primary_sub_division.append(cells[4].find(text=True))
        Notable_changes_from_2001_to_2011.append(cells[5].find(text=True))

IndexError: list index out of range

In [37]:
for row in right_table[1].findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==5:
        Area.append(cells[0].find(text=True))
        Population.append(cells[1].find(text=True))
        Area_size.append(cells[2].find(text=True))
        Density.append(cells[3].find(text=True))
        Primary_sub_division.append(cells[4].find(text=True))
        Notable_changes_from_2001_to_2011.append(cells[5].find(text=True))

IndexError: list index out of range